# Regular Genomics Project **RNA Localisation**


---

**Problem definition**:

TODO


### **1.** First steps with data

Firstly, we import several necessary packages and load in our data:

In [1]:
import pandas as pd
import numpy as np
import itertools
import os
import tensorflow as tf
import sys
from keras.utils import to_categorical
from collections import OrderedDict
from sklearn.model_selection import StratifiedKFold, KFold

In [2]:
# Initializing test set
np.random.seed(3)
data_org = pd.read_csv('final_data.csv')
test_data = data_org.sample(frac=0.1)
train_data = data_org.drop(test_data.index)

data_org

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,gene_id,gene_biotype,seq,struct,m6A_5UTR,m6A_CDS,m6A_3UTR
0,57.045409,35.456782,22.008215,12.355106,22.789983,24.241731,16.970436,29.348389,54.916891,ENSG00000000003,protein_coding,ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGATGCTTACAAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.914814,0.244517,ENSG00000000005,protein_coding,TGTGCACAGAAGTTATATACATATATGGGTATATCTATGTAACAAA...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
2,17.449430,34.151539,29.338431,22.237585,24.682767,43.612551,38.683963,35.678476,25.348560,ENSG00000000419,protein_coding,TACTTTATGCAAAAAAAAATATACATTTATTTATAGGTCTCAATAC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
3,3.830180,2.576734,5.737850,0.761343,2.786808,2.784356,3.382682,2.463676,2.819269,ENSG00000000457,protein_coding,TGACTTTCAAACCATTTTAATATTTCAAATATTCCAGAACAATCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,9,5
4,13.132915,8.782925,10.061390,3.012459,8.821250,6.721117,10.827253,8.005113,6.849962,ENSG00000000460,protein_coding,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.506809,0.075893,0.050764,0.000000,0.056586,0.000000,0.000000,ENSG00000281883,protein_coding,GGGAAGAAAGGAGCCTGACTCTTATGATGGAATAACCACAAATCAG...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,2,1
13806,0.105452,0.087130,0.171187,0.016101,0.079057,0.669947,0.171672,0.000000,0.254546,ENSG00000282034,protein_coding,GTGTCGGACGGCATGACAGGCAGCAATCCTGTGTCCCCTGCCTCAT...,"[0.37599998712539673, 0.0, 0.07500000298023224...",0,66,4
13807,0.037093,0.444844,0.425368,0.254467,0.323794,1.803249,1.435483,0.249590,0.483645,ENSG00000282827,protein_coding,ATGGCGGGGACCTCCGCGCCAGGCAGCAAGAGGCGGAGCGAGCCCC...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,0
13808,0.000000,0.519421,2.119115,0.360450,0.887939,0.345021,1.274465,0.741954,0.460649,ENSG00000282936,protein_coding,CTACTGGAACGCCCCCCTCAATCTAGCCTCCCCCACATAACTCTCT...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0,2


In [3]:
sum_vec = data_org.iloc[:, :9].sum(axis=1)
data2 = data_org.iloc[:, :9].divide(sum_vec, axis='index')
data = pd.concat([data2, data_org['seq']], axis=1)
data

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,seq
0,0.207338,0.128871,0.079991,0.044906,0.082833,0.088109,0.061681,0.106670,0.199601,ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGATGCTTACAAC...
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.922605,0.077395,TGTGCACAGAAGTTATATACATATATGGGTATATCTATGTAACAAA...
2,0.064346,0.125935,0.108187,0.082002,0.091019,0.160823,0.142649,0.131566,0.093474,TACTTTATGCAAAAAAAAATATACATTTATTTATAGGTCTCAATAC...
3,0.141112,0.094932,0.211394,0.028049,0.102672,0.102581,0.124625,0.090767,0.103868,TGACTTTCAAACCATTTTAATATTTCAAATATTCCAGAACAATCCC...
4,0.172315,0.115240,0.132014,0.039526,0.115743,0.088187,0.142063,0.105034,0.089878,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...
...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.734451,0.109981,0.073566,0.000000,0.082002,0.000000,0.000000,GGGAAGAAAGGAGCCTGACTCTTATGATGGAATAACCACAAATCAG...
13806,0.067811,0.056029,0.110082,0.010354,0.050838,0.430809,0.110393,0.000000,0.163685,GTGTCGGACGGCATGACAGGCAGCAATCCTGTGTCCCCTGCCTCAT...
13807,0.006797,0.081510,0.077941,0.046627,0.059330,0.330415,0.263028,0.045733,0.088620,ATGGCGGGGACCTCCGCGCCAGGCAGCAAGAGGCGGAGCGAGCCCC...
13808,0.000000,0.077421,0.315861,0.053726,0.132350,0.051426,0.189963,0.110591,0.068661,CTACTGGAACGCCCCCCTCAATCTAGCCTCCCCCACATAACTCTCT...


In [4]:
# One hot encode the 'seq' attribute of the above table
mapping = { 
            'A': 0,
            'C': 1,
            'G': 2,
            'T': 3
            }
one_hot_encode_lam = lambda seq: to_categorical([mapping[x] for x in seq])
data_one_hot_karas = data['seq'].apply(one_hot_encode_lam)

data_one_hot_karas

# Now just injecting this modified 'seq' back into the pandas frame 
data_new_karas =  pd.concat([data.iloc[:, :9], data_one_hot_karas], axis=1)

data_new_karas

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,seq
0,0.207338,0.128871,0.079991,0.044906,0.082833,0.088109,0.061681,0.106670,0.199601,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [..."
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.922605,0.077395,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [..."
2,0.064346,0.125935,0.108187,0.082002,0.091019,0.160823,0.142649,0.131566,0.093474,"[[0.0, 0.0, 0.0, 1.0], [1.0, 0.0, 0.0, 0.0], [..."
3,0.141112,0.094932,0.211394,0.028049,0.102672,0.102581,0.124625,0.090767,0.103868,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [..."
4,0.172315,0.115240,0.132014,0.039526,0.115743,0.088187,0.142063,0.105034,0.089878,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [..."
...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.734451,0.109981,0.073566,0.000000,0.082002,0.000000,0.000000,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [..."
13806,0.067811,0.056029,0.110082,0.010354,0.050838,0.430809,0.110393,0.000000,0.163685,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."
13807,0.006797,0.081510,0.077941,0.046627,0.059330,0.330415,0.263028,0.045733,0.088620,"[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."
13808,0.000000,0.077421,0.315861,0.053726,0.132350,0.051426,0.189963,0.110591,0.068661,"[[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."


### **2** |  Importing models...

We use as baseline model the [RNATracker](https://github.com/HarveyYan/RNATracker/blob/master/Models/cnn_bilstm_attention.py) model and a CNN model.


In [5]:
# Import NN
# from RNAtracker import RNATracker
from cnn_model import CNN_MLRG

# Set paths for model output
try:
    os.mkdir('model_outputs/')
except Exception as e:
    print("Output folder already exists")

model_output_folder = 'model_outputs/'

Output folder already exists


### SIMPLE CNN ARCHITECTURE

In [24]:
# TODO: do a normal CNN
import keras
from keras.layers import Conv1D, Dense, Flatten, MaxPooling1D, Dropout

def compute_cnn():
    model = keras.Sequential()

    model.add(Conv1D(64, kernel_size=3, activation='relu',input_shape=(34256, 4)))
    model.add(Dropout(.2))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
    model.add(Dropout(.2))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())

    model.add(Dense(9, activation="softmax"))

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

    model.summary()

    return model

In [26]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

print("Start training...")

from GeneDataLoader import GeneDataLoader

folds_total = 5

kf = KFold(n_splits=folds_total, shuffle=True, random_state=42)
folds = kf.split(np.zeros(data_new_karas.shape[0]))

for i, (train_indices, test_indices) in enumerate(folds):
    print('Evaluating KFolds {}/{}'.format(str(i + 1), str(folds_total)))

    model = compute_cnn()

    training_data_loader = GeneDataLoader(data_table=data_new_karas.iloc[train_indices], batch_size=64, shuffle=True)

    validation_data_loader = GeneDataLoader(data_table=data_new_karas.iloc[test_indices], batch_size=64, shuffle=True)

    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_output_folder + f"cnn{i}.h5",
                            monitor='val_accuracy', verbose=1,
                            save_best_only=True, mode='max')

    callbacks_list = [checkpoint]

    history = model.fit(training_data_loader, epochs=15, callbacks=callbacks_list, validation_data=validation_data_loader)

    model.load_weights(f"model_outputs/cnn{i}.h5")

    results = model.evaluate(validation_data_loader)
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

    tf.keras.backend.clear_session()

print("Training done!")

Start training...
Evaluating KFolds 1/5
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_15 (Conv1D)          (None, 34254, 64)         832       
                                                                 
 dropout_14 (Dropout)        (None, 34254, 64)         0         
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 17127, 64)        0         
 g1D)                                                            
                                                                 
 conv1d_16 (Conv1D)          (None, 17127, 64)         12352     
                                                                 
 dropout_15 (Dropout)        (None, 17127, 64)         0         
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 8563, 64)         0         
 g1D)         

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_5/dense_5/BiasAdd' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\iusti\anaconda3\envs\reggen\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\iusti\anaconda3\envs\reggen\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\iusti\anaconda3\envs\reggen\Lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\iusti\anaconda3\envs\reggen\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\iusti\anaconda3\envs\reggen\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "C:\Users\iusti\anaconda3\envs\reggen\Lib\asyncio\b